<a href="https://colab.research.google.com/github/macdinh99/LSTM_GRU_Hydrid/blob/main/GRUvsLSTM_hydrid_Ver3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Buoc 1: Khai bao thu vien lam viec

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import math
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
#Cac lop xay dung mo hinh
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.models import Sequential
from keras.layers import LSTM,GRU
from keras.layers import Dropout
from keras.layers import Dense
#Kiem tra do chinh xac
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance

Buoc 2: Doc du lieu

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
#Doc du lieu tu csv
df=pd.read_csv('/content/drive/MyDrive/VNindex_Ver4.csv')

In [ ]:
df.info()

Buoc 3: Mo ta va dinh dang du lieu

In [57]:
#Dinh dang cau truc theo time series
df["Date"]=pd.to_datetime(df.Date,format="%m/%d/%Y")

In [58]:
#Dinh dang du lieu kieu Object thanh Float
df["Price"] = df["Price"].str.replace(",", "").astype(float)

In [59]:
df = df.sort_values('Date', ascending=True)

In [ ]:
print("Starting date: ",df.iloc[0][0])
print("Ending date: ", df.iloc[-1][0])
print("Duration: ", df.iloc[-1][0]-df.iloc[0][0])

In [ ]:
df.describe()

In [ ]:
df.info()

Buoc 4: Chuan bi xu ly du lieu

In [ ]:
#Phan lay va xu ly du lieu

In [62]:
df2=df.copy()

In [63]:
#Lay du lieu ngay va ket phien giao dich
df2=df2[['Date','Price']]

In [64]:
df2 = df2.sort_values('Date', ascending=True)
df2.index=df2.Date

In [ ]:
#Chuan hoa mo hinh
del df2['Date']
scaler=MinMaxScaler(feature_range=(0,1))
df2=scaler.fit_transform(np.array(df2).reshape(-1,1))
print(df2.shape)

In [ ]:
#Chia tap du lieu
training_size=int(len(df2)*0.80)
test_size=len(df2)-training_size
train_data,test_data=df2[0:training_size,:],df2[training_size:len(df2),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

In [67]:
#convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 50
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

In [ ]:
X_train
X_test

In [ ]:
temp_y_train=y_train
temp_y_test=y_test
f, (ax1,ax2) = plt.subplots(1, 2, figsize=(14,5))
ax1.hist(y_train, bins=20, color='blue', alpha=0.7)
ax1.set_xlabel("Giá trị", fontsize=12)
ax1.set_ylabel("Tần suất")
ax1.set_title("(a)")
ax2.hist(y_test, bins=20, color='green', alpha=0.7)
ax2.set_xlabel("Giá trị", fontsize=12)
ax2.set_ylabel("Tần suất")
ax2.set_title("(b)")
ax2.legend()

In [ ]:
#Ket thuc phan lay va xu ly du lieu

Buoc 5: Xay dung va huan luyen mo hinh

In [ ]:
#GRU: Xay dung mo hinh

In [71]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(128,return_sequences=True,input_shape=(time_step,1)))
model.add(GRU(64))
model.add(Dropout(0.25))
model.add(Dense(1))
model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
#GRU: Ket thuc xay dung mo hinh

In [ ]:
#LSTM: Xay dung mo hinh

In [17]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(64))
model.add(Dropout(0.25))
model.add(Dense(1))
model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
#LSTM: Ket thuc xay dung mo hinh

In [ ]:
#GRU-LSTM hydrid: Xay dung mo hinh

In [41]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(64,return_sequences=True,input_shape=(time_step,1)))
model.add(GRU(32,return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
#GRU-LSTM hydrid: Ket thuc xay dung mo hinh

In [ ]:
#LSTM-GRU hydrid: Xay dung mo hinh

In [18]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(64,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(GRU(64,return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
#LSTM-GRU hydrid: Ket thuc xay dung mo hinh

In [ ]:
#Huan luyen mo hinh

In [ ]:
start_time = dt.datetime.now()
history1 = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=50,verbose=1)
end_time = dt.datetime.now()
execution_time = end_time - start_time
print("Thời gian thực thi: ", execution_time)

In [ ]:
# Summarize history for loss
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(len(loss))
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.locator_params(axis='y', nbins=15, tight=True)
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
#Ket thuc huan luyen mo hinh

Buoc 6: Ap dung mo hinh

In [ ]:
#Ap dung mo hinh

In [ ]:
# Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

In [ ]:
#Do thi phan bo y_train va traim_predict

In [ ]:
f, (ax1,ax2) = plt.subplots(1, 2, figsize=(14,5))
ax1.hist(train_predict, bins=20, color='blue', alpha=0.7,label='Tập dự báo')
ax1.hist(temp_y_train, bins=20, color='pink', alpha=0.7,label='Tập huấn luyện')
ax1.set_xlabel("Giá trị", fontsize=12)
ax1.set_ylabel("Tần suất")
ax1.set_title("(a)")
ax2.hist(test_predict, bins=20, color='green', alpha=0.7, label='Tập dự báo')
ax2.hist(temp_y_test, bins=20, color='red', alpha=0.7, label='Tập giám sát')
ax2.set_xlabel("Giá trị", fontsize=12)
ax2.set_ylabel("Tần suất")
ax2.set_title("(b)")
ax1.legend()
ax2.legend()

In [34]:
# Transform back to original form
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1))
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [ ]:
# shift train predictions for plotting
close_stock = df.copy()
look_back=time_step
trainPredictPlot = np.empty_like(df2)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(df2)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df2)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Thực tế','Dự báo tập huấn luyện','Dự báo tập giám sát'])


plotdf = pd.DataFrame({'Date': close_stock['Date'],
                       'original_close': close_stock['Price'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['Date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Chỉ số VNIndex','Date': 'Thời gian'})
fig.update_layout(title_text='Biểu đồ so sánh chỉ số VNIndex thực tế và dự báo', title_x=0.5,
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Chỉ số VNIndex')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
#Ket thuc ap dung mo hinh

Buoc 7: Đo lường độ chính xác

In [ ]:
#Do luong

In [ ]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print('Train data MAPE:',mean_absolute_percentage_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data R2 score:", r2_score(original_ytest, test_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))
print('Test data MAPE:',mean_absolute_percentage_error(original_ytest,test_predict))

In [ ]:
#Ket thu do luong

Buoc 8: Du bao

In [109]:
#Ung dung mo hinh de bao 100 ngay tiep theo

In [37]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 50
while(i<pred_days):

    if(len(temp_input)>time_step):

        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))

        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)

        lst_output.extend(yhat.tolist())
        i=i+1

    else:

        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())

        lst_output.extend(yhat.tolist())
        i=i+1

print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  50


In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

In [ ]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(df2[len(df2)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Dự báo 50 ngày đầu tiên','Dự báo 50 ngày kế tiếp'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Chỉ số VNIndex','index': 'Mốc thời gian'}, color_discrete_sequence=['blue','red'])
fig.update_traces(line_color='blue', selector=dict(name='Dự báo 50 ngày đầu tiên'))
fig.update_traces(line_color='red', selector=dict(name='Dự báo 50 ngày kế tiếp'))
for trace, name in zip(fig.data, names):
    trace.name = name
fig.update_layout(title_text='Biểu đồ xu hướng 50 ngày đầu và 50 ngày tiếp theo', title_x=0.5,
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Chỉ số VNIndex')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=True)
fig.update_yaxes(showgrid=True)
fig.show()

In [ ]:
lstmdf=df2.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Chỉ số VNIndex'])

fig = px.line(lstmdf,labels={'value': 'Chỉ số VNIndex','index': 'Mốc thời gian'})
fig.update_layout(title_text='Biểu đồ xu hướng chỉ số VNIndex', title_x=0.5,
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Xu hướng')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
plt.savefig('dothi.png')
fig.show()

In [ ]:
#Ket thu ung dung mo hinh du bao 100 ngay tiep theo